In [5]:
import os
import sys
# import soundfile
import random
import pandas as pd
import torch
import torchaudio
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from datasets import Dataset
from sklearn.model_selection import KFold
from peft import LoraConfig, get_peft_model


In [16]:
### Load Charsiu model
charsiu_dir = '/home/cogsci-lasrlab/ForcedAlignment/programs/charsiu-main'
os.chdir(charsiu_dir)

sys.path.append('%s/src/' % charsiu_dir)

from Charsiu import charsiu_forced_aligner

In [ ]:
# add to requirements.in
# praatio, librosa, nltk, g2p_en, g2pM


  Using cached g2pM-0.1.2.5-py3-none-any.whl.metadata (4.0 kB)
Using cached g2pM-0.1.2.5-py3-none-any.whl (1.7 MB)
Note: you may need to restart the kernel to use updated packages.


In [17]:
# Set random seed for reproducibility
random.seed(1234)
torch.manual_seed(1234)

# Load participant sets CSV
participant_df = pd.read_csv("/home/cogsci-lasrlab/Documents/CSS_Capstone/lora_charsiu/participant_sets.csv")
training_ids = participant_df[participant_df['set'].str.lower() == 'training']['ParticipantID'].str.lower().tolist()
training_ids = [id.strip().lower() for id in training_ids]
eval_ids = participant_df[participant_df['set'].str.lower() == 'evaluation']['ParticipantID'].str.lower().tolist()

# Load csv of incorrect utterances
# Create set of included files for training
inclusion_df = pd.read_csv("/home/cogsci-lasrlab/Documents/CSS_Capstone/lora_charsiu/incorrect_utterances.csv")
incorrect_word_files = inclusion_df[inclusion_df['Subject inclusion'].str.lower() == 'include']['Filename'].tolist()
incorrect_word_files = set([f.lower() for f in incorrect_word_files])  # normalize case

In [ ]:
# extract filepath, participant info, and word from KT1 data
base_dir = "/home/cogsci-lasrlab/Documents/CSS_Capstone/KT1_data"
data = []
included_count = 0
excluded_count = 0
num_files = 0
num_train = 0
num_eval = 0

for folder in os.listdir(base_dir):
    folder_path = os.path.join(base_dir, folder)
    if os.path.isdir(folder_path):
        for file in os.listdir(folder_path):
            if file.endswith('.wav') and "participant_" in file:
                base = file[:-4]  # remove '.wav'
                before, word = base.split("participant_")
                    
                # Remove 'K1' and researcher number before 'participant_'
                ppt_id = before.replace("K1", "")[:-1]
                ppt_id_clean = ppt_id.strip().lower() 
                
                # handle missing participant
                if "rri0" in file.lower():
                    ppt_id_clean = "rri0"
                num_files += 1

                # Check if file has correct word
                if file.lower() not in incorrect_word_files:
                    included_count += 1
                    # Only include training data
                    if ppt_id_clean.lower() in training_ids:
                        num_train += 1
                        data.append({
                            "file_name": os.path.join(folder_path, file),
                            "ppt_id": ppt_id,
                            "transcription": word
                            })
                            
                            # Write .txt file
#                             txt_path = os.path.join(folder_path, f"{os.path.splitext(file)[0]}.txt")
#                             with open(txt_path, 'w') as f:
#                                 f.write(word)
                                
                    #Count number of excluded file
                    else:
                        num_eval += 1
                else:
                    excluded_count += 1

# Convert to DataFrame
df = pd.DataFrame(data)

# Report
print(f"{num_files} total files.") # should be 1728
print(f"{included_count} files were included (both training and evaluation).") # should be 1591
print(f"{num_train} files are used for training.") # should be 1192
print(f"{num_eval} files are used fr evaluation.") # should be 399
print(f"{excluded_count} files were excluded due to incorrect word.") # should be 137

1728 total files.
1591 files were included (both training and evaluation).
1192 files are used for training.
399 files are used fr evaluation.
137 files were excluded due to incorrect word.


In [19]:
# Load Wav2Vec model and processor
base_model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
# Configure LoRA arguments
lora_config = LoraConfig(
    r=8, # can change rank, r=8 is most common
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    target_modules=["q_proj", "v_proj"] # attention layers
)

# Add LoRA layers to Wav2Vec2 model
model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

trainable params: 294,912 || all params: 94,691,232 || trainable%: 0.3114


In [42]:
def preprocess(example):
    # Load audio
    waveform, sr = torchaudio.load(example["file_name"])
    
    # Resample
    if sr != 16000:
        resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=16000)
        waveform = resampler(waveform)

    # Flatten list of waveforms for correct dimensionality
    waveform = waveform.squeeze(0)  # shape becomes [samples]

    # Use the feature extractor (not full processor) — output is a dict with lists
    inputs = processor.feature_extractor(
        waveform,
        sampling_rate=16000,
        padding=False, # ensures files are the same size
        return_attention_mask=True, # tells the model what is actually audio and what is padding
        return_tensors=None  
    )

    # Flatten input_values from [[...]] → [...] for correct dimensionality
    input_values = inputs["input_values"][0]
    attention_mask = inputs["attention_mask"][0]

    # Tokenize word transcripts (label_ids)
    with processor.as_target_processor():
        label_ids = processor.tokenizer(
            example["transcription"].upper(), # upper to ensure correct tokenization
            return_tensors=None,
            padding=False,
            truncation=True
        ).input_ids

    return {
        # keep all inputs lists
        "input_values": input_values,
        "attention_mask": attention_mask,
        "labels": label_ids  
    }

In [43]:
# Convert data from pandas dataframe to transformers dataset
# Preprocesses data for training
dataset = Dataset.from_pandas(df)
processed_dataset = dataset.map(preprocess, remove_columns=["file_name", "ppt_id", "transcription"])

Map:   0%|          | 0/1192 [00:00<?, ? examples/s]

/home/cogsci-lasrlab/Documents/CSS_Capstone/lora_charsiu/.venv/lib/python3.12/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 1192/1192 [00:18<00:00, 64.97 examples/s]


In [49]:
# See if tokenization is correct
def tokenize_transcription(text):
    with processor.as_target_processor():
        return processor.tokenizer(
            text.upper(),
            return_tensors=None,
            padding=True,
            truncation=True
        ).input_ids
    
df["labels"] = df["transcription"].apply(tokenize_transcription)
df

/home/cogsci-lasrlab/Documents/CSS_Capstone/lora_charsiu/.venv/lib/python3.12/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


,file_name,ppt_id,transcription,labels
0,/home/cogsci-lasrlab/Documents/CSS_Capstone/KT...,cb01,sun,"[12, 16, 9]"
1,/home/cogsci-lasrlab/Documents/CSS_Capstone/KT...,lt04,chick,"[19, 11, 10, 19, 26]"
2,/home/cogsci-lasrlab/Documents/CSS_Capstone/KT...,cb10,sun,"[12, 16, 9]"
3,/home/cogsci-lasrlab/Documents/CSS_Capstone/KT...,cb10,chick,"[19, 11, 10, 19, 26]"
4,/home/cogsci-lasrlab/Documents/CSS_Capstone/KT...,cb01,ramp,"[13, 7, 17, 23]"
...,...,...,...,...
1187,/home/cogsci-lasrlab/Documents/CSS_Capstone/KT...,AM16,window,"[18, 10, 9, 14, 8, 18]"
1188,/home/cogsci-lasrlab/Documents/CSS_Capstone/KT...,lm01,ship,"[12, 11, 10, 23]"
1189,/home/cogsci-lasrlab/Documents/CSS_Capstone/KT...,pv04,sun,"[12, 16, 9]"
1190,/home/cogsci-lasrlab/Documents/CSS_Capstone/KT...,EB21,red,"[13, 5, 14]"


In [45]:
# from huggingface transformers
# Ensures that dataset is properly formatted for training

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch


In [54]:
from transformers import TrainingArguments, Trainer

# For testing
training_args = TrainingArguments(
    output_dir="/home/cogsci-lasrlab/Documents/CSS_Capstone/lora_charsiu/lora_test", # use absolute path
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    eval_strategy="no",
    save_strategy="no",
    learning_rate=5e-4,
    weight_decay=0.0, # L2 regularization, lower if the loss 
    num_train_epochs=1, # make this larger
    fp16=False,
    gradient_accumulation_steps=5, # experiment with making this larger (larger = faster)
    # logging_dir="./logs_test",
    logging_steps=10,
    report_to="none",
    label_names=["labels"]
)

# # For final training setup
# training_args = TrainingArguments(
#     output_dir="./lora_charsiu_finetuned", # use absolute path
#     per_device_train_batch_size=2,
#     per_device_eval_batch_size=2,
#     eval_strategy="epoch",
#     save_strategy="epoch",
#     learning_rate=5e-4,
#     weight_decay=0.01, # L2 regularization
#     num_train_epochs=10, # make this larger
#     fp16=True,
#     gradient_accumulation_steps=3, # experiment with making this larger (larger = faster)
#     logging_dir="./logs",
#     report_to="tensorboard",
#     label_names=["labels"]
# )

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_dataset,
    data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)
)


In [55]:
trainer.train()
print("Done training.")
model.save_pretrained("/home/cogsci-lasrlab/Documents/CSS_Capstone/lora_charsiu/lora_test") # use absolute path
processor.save_pretrained("/home/cogsci-lasrlab/Documents/CSS_Capstone/lora_charsiu/lora_test") # use absolute path
# REVISIT TRAINING
# zero training loss (again)

# for final training
# trainer.train()
# print("Done training.")
# model.save_pretrained("./lora_charsiu_finetuned") # use absolute path
# processor.save_pretrained("./lora_charsiu_finetuned") # use absolute path

# output should be:
## pytorch_model.bin
## preprocessor_config.json

# have a more gradient measure of assessment
## sum of squared error relative to manual alignments
## makes it more obvious when the model is doing bad


## make a for loop that training multiple models
# for lora_rank in [2. 4. 8, etc.]:
    # train lora adapted model
    # infer on evaluation set
    # create dataframe of assessment measures
    # graph error by rank

# report IRR between humans
# report IRR between models vs humans, treat model as another human

/home/cogsci-lasrlab/Documents/CSS_Capstone/lora_charsiu/.venv/lib/python3.12/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Step,Training Loss
10,0.000000
20,0.000000
30,0.000000
40,0.000000
50,0.000000
60,0.000000


KeyboardInterrupt: 

**TO DO:**
- Train model
    - Try Wav2Vec training from article
- Make a new folder containing only evaluation set audio files
- **Write new script:**
    - Run inference on models (lora with different ranks) for evaluation set
    - Calculate error measurements (relative to human gold standard, see picture from Will's OH)
    - Sum errors and compare across models

-**As another metric of model performance:** Report IRR humans vs. humans, humans vs. best model